In [1]:
from os import listdir
from os.path import join,isfile

my_path = "F:/Datasets/catsvsdogs/images/"
file_names = [f for f in listdir(my_path) if isfile(join(my_path,str(f)))]
print(f"{len(file_names)} images loaded")

3002 images loaded


In [2]:
import cv2
import numpy as np
import sys
import os
import shutil

dog_count = 0
cat_count = 0
training_size = 1000
test_size = 500
training_images = []
test_images = []
test_labels = []
training_labels = []
size = 150
dog_dir_train = "F:/Datasets/catsvsdogs/train/dogs/"
cat_dir_train = "F:/Datasets/catsvsdogs/train/cats/"
dog_dir_val = "F:/Datasets/catsvsdogs/validation/dogs/"
cat_dir_val = "F:/Datasets/catsvsdogs/validation/cats/"

def make_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

make_dir(dog_dir_train)
make_dir(cat_dir_train)
make_dir(dog_dir_val)
make_dir(cat_dir_val)

def getZeros(number):
    if number>10 and number<100:
        return "0"
    if number<10:
        return "00"
    else:
        return ""
for i ,file in enumerate(file_names):

     if file_names[i][0] == "d":
        dog_count+=1
        image = cv2.imread(my_path+str(file_names[i]))
        image = cv2.resize(image, (size,size), interpolation=cv2.INTER_AREA)
        if dog_count <= training_size:
            training_images.append(image)
            training_labels.append(1)
            zeros = getZeros(dog_count)
            cv2.imwrite(dog_dir_train+"dog"+str(zeros)+str(dog_count)+'.jpg',image)
        if dog_count > training_size and dog_count <= training_size+test_size:
            test_images.append(image)
            test_labels.append(1)
            zeros = getZeros(dog_count-1000)
            cv2.imwrite(dog_dir_val+"dog"+str(zeros)+str(dog_count-1000)+'.jpg',image)
     if file_names[i][0] == "c":
        cat_count+=1
        image = cv2.imread(my_path+str(file_names[i]))
        image = cv2.resize(image, (size,size), interpolation=cv2.INTER_AREA)
        if cat_count <= training_size:
            training_images.append(image)
            training_labels.append(0)
            zeros = getZeros(cat_count)
            cv2.imwrite(cat_dir_train+"cat"+str(zeros)+str(cat_count)+'.jpg',image)
        if cat_count > training_size and cat_count <= training_size+test_size:
            test_images.append(image)
            test_labels.append(0)
            zeros = getZeros(cat_count-1000)
            cv2.imwrite(cat_dir_val+"cat"+str(zeros)+str(cat_count-1000)+'.jpg',image)

     if dog_count == training_size + test_size and cat_count == training_size + test_size:
        break
print("Training and test data extraction completed")

Training and test data extraction completed


In [3]:
np.savez('F:/Datasets/catsvsdogs/cats_vs_dog_training_data.npz',np.array(training_images))
np.savez('F:/Datasets/catsvsdogs/cats_vs_dog_training_labels.npz',np.array(training_labels))
np.savez('F:/Datasets/catsvsdogs/cats_vs_dog_test_data.npz',np.array(test_images))
np.savez('F:/Datasets/catsvsdogs/cats_vs_dog_test_labels.npz',np.array(test_labels))

In [4]:
def load_data_training_and_test(path,datasetname):
    npz_file = np.load(path+datasetname+"_training_data.npz")
    train = npz_file['arr_0']
    npz_file = np.load(path+datasetname+"_training_labels.npz")
    train_labels = npz_file['arr_0']
    npz_file = np.load(path+datasetname+"_test_data.npz")
    test = npz_file['arr_0']
    npz_file = np.load(path+datasetname+"_test_labels.npz")
    test_labels = npz_file['arr_0']

    return (train,training_labels), (test,test_labels)

In [5]:
for i in range(1,11):
    random = np.random.randint(0, len(training_images))
    cv2.imshow("Image_"+str(i),training_images[random])
    if training_labels[random] == 0:
        print(str(i)+"-Cat")
    else:
        print(str(i)+"-Dog")
    cv2.waitKey(0)
cv2.destroyAllWindows()

1-Dog
2-Dog
3-Cat
4-Cat
5-Cat
6-Dog
7-Dog
8-Dog
9-Dog
10-Dog


In [15]:
(x_train,y_train),(x_test,y_test) = load_data_training_and_test("F:/Datasets/catsvsdogs/","cats_vs_dog")
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = x_train/255
x_test = x_test/255
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2000, 150, 150, 3)
(2000, 1)
(1000, 150, 150, 3)
(1000, 1)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten



In [19]:
image_shape = (x_train[0].shape[0],x_train[1].shape[0],3)
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)            

In [22]:
model.fit(x_train,y_train,batch_size= 16 ,epochs =25,
          validation_data=(x_test,y_test),shuffle=True)

Train on 2000 samples, validate on 1000 samples
Epoch 1/25
2000/2000 [==============================] - 10s 5ms/sample - loss: 0.6947 - accuracy: 0.5295 - val_loss: 0.6918 - val_accuracy: 0.5000

In [23]:
model.evaluate(x_test,y_test)

1000/1000 [==============================] - 1s 525us/sample - loss: 1.6185 - accuracy: 0.7030


[1.6185113544464111, 0.703]

In [24]:
def draw_test(name,pred,input_im):
    Black = []
    if pred =="[0]":
        pred == "cat"
    if pred =="[1]":
        pred == "dog"
    expand_image = cv2.copyMakeBorder(input_im,0, 0, 0,imageL.shape[0],cv2.BORDER_CONSTANT,value=Black )
    cv2.putText(expand_image,str(pred), (252,70),cv2.FONT_HERSHEY_COMPLEX_SMALL, 4, (0,255,0), 2)
    cv2.imshow(name,expand_image)

for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]
    imageL = cv2.resize(input_im,None,fx=2,fy=2,interpolation=cv2.INTER_CUBIC)
    cv2.imshow("Test Image",imageL)
    input_im = input_im.reshape(1,150,150,3)
    res = str(model.predict_classes(input_im,1,verbose=0)[0])

    draw_test("prediction",res,imageL)
    cv2.waitKey(0)

cv2.destroyAllWindows()
